In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"D:\Machine Learning\final_preprocessed_data.csv")

In [2]:
df.head()

,state_code,dist_code,type_name,purpose_name,disp_name,Judge_Labels,Lag,Total_Days
0,1,1,1943,2975,33,55,0.0,37.0
1,1,1,1943,3315,52,55,0.0,31.0
2,1,1,1943,5877,52,55,0.0,31.0
3,1,1,1943,840,52,55,0.0,31.0
4,1,1,1943,840,5,55,0.0,8.0


In [3]:
df['type_name'].value_counts()

981     513845
977     426036
736     328940
1892    294604
853     275986
         ...  
5390      2071
3090      2046
5297      1922
4975      1572
4947      1036
Name: type_name, Length: 500, dtype: int64

In [4]:
top_100_positions = df['type_name'].value_counts().head(100).index
df = df[df['type_name'].isin(top_100_positions)]

In [5]:
top_40_positions = df['Judge_Labels'].value_counts().head(40).index
df = df[df['Judge_Labels'].isin(top_40_positions)]

In [6]:
df.shape

(8980039, 8)

In [7]:
df[df['type_name']==977]['Total_Days'].value_counts()

0.0      155761
1.0        5890
2.0        4301
8.0        2818
3.0        2709
          ...  
820.0         1
935.0         1
929.0         1
882.0         1
838.0         1
Name: Total_Days, Length: 837, dtype: int64

In [8]:
import pandas as pd

for value in df['type_name'].unique():
    specific_type_name = value
    specific_type_data = df[df['type_name'] == specific_type_name]
    Q1 = specific_type_data['Total_Days'].quantile(0.25)
    Q3 = specific_type_data['Total_Days'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.3 * IQR
    upper_bound = Q3 + 1.3 * IQR
    df = df[(df['type_name'] != specific_type_name) | ((df['type_name'] == specific_type_name) & (df['Total_Days'] >= lower_bound) & (df['Total_Days'] <= upper_bound))]


In [9]:
df.head()

,state_code,dist_code,type_name,purpose_name,disp_name,Judge_Labels,Lag,Total_Days
0,1,1,1943,2975,33,55,0.0,37.0
1,1,1,1943,3315,52,55,0.0,31.0
2,1,1,1943,5877,52,55,0.0,31.0
3,1,1,1943,840,52,55,0.0,31.0
4,1,1,1943,840,5,55,0.0,8.0


In [27]:
bins1 = [-1,20, 40, 80, 200, float('inf')]
labels1 = [1, 2, 3, 4,5]

df['Lag_grp']= pd.cut(df['Lag'],bins = bins1, labels = labels1)
df['Lag_grp'] = df['Lag_grp'].astype('int')

In [28]:
df['Lag_grp'].value_counts()

1    6066444
2     786441
3     645367
4     389575
5      89786
Name: Lag_grp, dtype: int64

In [29]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


In [30]:
X_lag = df[['state_code','dist_code', 'type_name','Judge_Labels']]
y_lag = df['Lag_grp']

X_train_lag, X_test_lag, y_train_lag, y_test_lag = train_test_split(X_lag, y_lag, test_size=0.2)


In [31]:
best_params = {
    'objective': 'regression',
    'num_leaves': 70,
    'n_estimators': 500,
    'learning_rate': 0.3,
    'feature_fraction': 0.8,
    'boosting_type': 'gbdt',
    'bagging_freq': 4,
    'bagging_fraction': 0.8,
    'force_col_wise':True
}


train_data = lgb.Dataset(X_train_lag, label=y_train_lag)


In [32]:
model_lag = lgb.train(params=best_params, train_set=train_data, num_boost_round=200)
y_pred_lag = model_lag.predict(X_test_lag)

C:\Users\dell\anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 246
[LightGBM] [Info] Number of data points in the train set: 6382090, number of used features: 4
[LightGBM] [Info] Start training from score 1.451669


In [33]:
mse = mean_squared_error(y_test_lag, y_pred_lag)
r2 = r2_score(y_test_lag, y_pred_lag)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 0.4324188430502841
R-squared: 0.484498898265157


In [34]:
y_pred = y_pred_lag
for i in range(len(y_pred)):
    if y_pred_lag[i] < 0:
        y_pred[i] = 1
    elif y_pred[i] >= 0 and y_pred[i] < 1:
        y_pred[i] = 1
    elif y_pred[i] >= 1 and y_pred[i] < 1.5:
        y_pred[i] = 1
    elif y_pred[i] >= 1.5 and y_pred[i] < 2:
        y_pred[i] = 2
    elif y_pred[i] >= 2 and y_pred[i] < 2.5:
        y_pred[i] = 2
    elif y_pred[i] >= 2.5 and y_pred[i] < 3:
        y_pred[i] = 3
    elif y_pred[i] >= 3 and y_pred[i] < 3.5:
        y_pred[i] = 3
    elif y_pred[i] >= 3.5 and y_pred[i] < 4:
        y_pred[i] = 4
    elif y_pred[i] >= 4 and y_pred[i] <= 4.5:
        y_pred[i] = 4
    elif y_pred[i] > 4.5:
        y_pred[i] = 5

In [35]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test_lag,y_pred_lag)
acc

0.738848014099452

In [36]:
pred = pd.DataFrame(y_pred_lag)
pred.value_counts()

1.0    1119642
2.0     323584
3.0     136281
4.0      14795
5.0       1221
dtype: int64

In [38]:
y_test_lag.value_counts()

1    1212722
2     157118
3     129630
4      78141
5      17912
Name: Lag_grp, dtype: int64

In [40]:
import pickle
with open('model_lag_grp.pkl','wb') as f:
    pickle.dump(model_lag,f)